<a href="https://colab.research.google.com/github/NhutTien0905/NhutTien0905/blob/main/flower_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/file/iris.csv")
data.head()

,senpal lenght,senpal width,pental length,pental width,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
data = data.sample(frac = True)
data

,senpal lenght,senpal width,pental length,pental width,type
93,5.0,2.3,3.3,1.0,Iris-versicolor
33,5.5,4.2,1.4,0.2,Iris-setosa
42,4.4,3.2,1.3,0.2,Iris-setosa
46,5.1,3.8,1.6,0.2,Iris-setosa
81,5.5,2.4,3.7,1.0,Iris-versicolor
...,...,...,...,...,...
26,5.0,3.4,1.6,0.4,Iris-setosa
132,6.4,2.8,5.6,2.2,Iris-virginica
89,5.5,2.5,4.0,1.3,Iris-versicolor
85,6.0,3.4,4.5,1.6,Iris-versicolor


In [4]:
Ys = data[["type"]].to_numpy()
Xs = data.drop(columns = "type").to_numpy()
print(Ys.shape)
print(Xs.shape)

(150, 1)
(150, 4)


In [5]:
def TypetoVec(typee,maxLength = 3):
    vec = np.zeros(maxLength)
    if typee == "Iris-setosa":
        vec[0] = 1
    elif typee == "Iris-versicolor":
        vec[1] = 1
    else:
        vec[2] = 1
    return vec

TypetoVec("Iris-versicolor")

array([0., 1., 0.])

In [8]:
trainX = []
trainY = []
labels = ["Iris-setosa","Iris-versicolor","Iris-virginica"]

trainPair = list(zip(Xs,Ys))
random.shuffle(trainPair)
nSample = int(0.9*len(Xs))
for i in range(nSample):
    x,y = trainPair[i]
    x = x.reshape(len(x),1)
    trainX.append(x)
    #y = TypetoVec(y)
    trainY.append(y)

trainX[0].shape


(4, 1)

In [7]:
def create(layerArr,inputDim):
    weights = []
    biases = []
    activ = []

    for i in range(len(layerArr)):
        nodeNum = layerArr[i][0]
        wLayer = []
        biasLay = []
        if i ==0 :
            lastLayer = inputDim
        else:
            lastLayer = layerArr[i-1][0]

        for n in range(nodeNum):
            wNode = []
            for l in range(lastLayer):
                wNode.append(1)
            wLayer.append(wNode)
            biasLay.append(0)
        
        weights.append(wLayer)
        biases.append(biasLay)
        activ.append(layerArr[i][1])
    return weights,biases,activ

In [9]:
layerArr = [[len(labels),"sigmoid"]]
inputDim = 4
neural = create(layerArr,inputDim)
print(neural[0])

[[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]]


In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoidDeri(x):
    return x*(1-x)

def relu(x):
    if x < 0:
        return 0
    else:
        return x

In [11]:
def predictRatio(data,neural):
    weights = neural[0]
    biases = neural[1]
    activ = neural[2]
    layerNum = len(weights)

    for l in range(layerNum):
        data = np.dot(weights[l],data)
        for t in range(len(data)):
            data[t] += biases[l][t]
        if activ[l] == "sigmoid":
            data = sigmoid(data)
        elif activ[l] == "relu":
            data = relu(data)
        else:
            data = sigmoid(data)
            print("sai r")
    return data


In [12]:
pre = predictRatio(trainX,neural)
print(pre[0][9])

[0.99999985]


In [13]:
def predict(data,neural):
    data = predictRatio(data,neural)
    classNum = len(data)

    highestClass = None
    highestProb = -1

    for i in range(classNum):
        if highestClass == None:
            highestClass = i
            highestProb = data[i]
        elif data[i] > highestProb:
            highestClass = i
            highestProb = data[i]
    return highestClass,highestProb

In [14]:
print(predict(trainX[0],neural))

(0, array([0.9999999]))


In [15]:
def train(X,Y,labels,neural,epochs = 1000):
    for epoch in range(epochs):
        for d in range(len(X)):
            pre = predictRatio(X[d],neural)
            truePre = []
            for i in range(len(labels)):
                truePre.append(0)
            truePre[labels.index(Y[d])] = 1

            errors = []
            for t in range(len(pre)):
                errors.append(truePre[t] - pre[t])
            adjustDeriv = errors * sigmoidDeri(pre)

            for k in range(len(adjustDeriv)):
                adjustment = np.dot(X[d],adjustDeriv[k])
                neural[0][0][k] += adjustment

    return neural

In [16]:
neural = train(trainX,trainY,labels,neural,epochs = 10000)

In [17]:
trainX[00]

array([[5.8],
       [2.8],
       [5.1],
       [2.4]])

In [18]:
print(predict(np.array([6.2,3.25,4.78,1.23]),neural))

(1, 0.25847975371050885)


In [19]:
print(predict(np.array([6.2,3.25,4.9,1.9]),neural))

(2, 0.013582751739999564)


In [20]:
print(predict(np.array([5.23,3.14,1.34,0.57]),neural))

(0, 0.9997100573756819)
